Paso 1: Carga del Dataset
1. Importar Pandas y NumPy: Son las librerías fundamentales para manejar datos (pandas) y realizar cálculos numéricos eficientes (numpy).

2. Cargar el Archivo: Usamos pd.read_csv() para leer el archivo del AI4I 2020 Predictive Maintenance Dataset. Asumiremos que el archivo se llama ai4i2020.csv y se encuentra en la misma carpeta que tu Jupyter Notebook.

3. Mostrar Encabezado: Usamos .head() para ver las primeras 10 filas y confirmar que la carga fue exitosa y para empezar a inspeccionar el formato de los datos.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("./ai4i2020.csv")

df.head(10)

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
5,6,M14865,M,298.1,308.6,1425,41.9,11,0,0,0,0,0,0
6,7,L47186,L,298.1,308.6,1558,42.4,14,0,0,0,0,0,0
7,8,L47187,L,298.1,308.6,1527,40.2,16,0,0,0,0,0,0
8,9,M14868,M,298.3,308.7,1667,28.6,18,0,0,0,0,0,0
9,10,M14869,M,298.5,309.0,1741,28.0,21,0,0,0,0,0,0


Paso 2: Limpieza de nombres de columnas y Análisis Exploratorio de Datos (EDA)
1. Crea una herramienta que toma un nombre de columna, elimina espacios al inicio/final, y reemplaza secuencias de múltiples espacios con un solo espacio.
2. df.info(): Imprime un resumen de cuántos valores no nulos hay en cada columna y el tipo de dato (Dtype).
3. df.describe(): Calcula estadísticas básicas (media, min, max, desviación estándar) para las columnas numéricas.
4. df[NOMBRE_OBJETIVO].value_counts()

In [2]:
# Función para limpiar nombres: reemplaza múltiples espacios por uno y elimina espacios al inicio/final
def limpiar_nombre_columna(col):
    # ' '.join(col.split()) elimina cualquier secuencia de espacios y la reemplaza por un solo espacio.
    col = ' '.join(col.split())
    return col.strip()

# Aplicar la limpieza a todos los nombres de columna
df.columns = [limpiar_nombre_columna(col) for col in df.columns]
""""
print("--- Nombres de columnas después de la limpieza ---")
print(df.columns.tolist())
"""

# --- 2.2 ANÁLISIS DE TIPOS DE DATOS Y ESTADÍSTICAS ---

print("\n--- 2.2.1 Información General del DataFrame ---")
# Muestra el tipo de dato y la cuenta de valores no nulos (verificar si hay NaNs)
df.info()

print("\n--- 2.2.2 Estadísticas Descriptivas de Variables Numéricas ---")
# Muestra estadísticas como media, desviación estándar, min/max.
print(df.describe())


# --- 2.3 ANÁLISIS DE LA VARIABLE OBJETIVO ---

print(f"\n--- 2.3.1 Conteo de la Variable Objetivo ('Machine failure') ---")
# Analiza el balance de la variable objetivo (0 = No Fallo, 1 = Fallo)
conteo_fallos = df['Machine failure'].value_counts()
print(conteo_fallos)

# Calcular el porcentaje de fallos
porcentaje_fallos = (conteo_fallos.get(1, 0) / df.shape[0]) * 100
print(f"\nPorcentaje de Fallo de Máquina (Clase 1): {porcentaje_fallos:.2f}%")
print("Este bajo porcentaje confirma un problema de desbalance de clases.")


--- 2.2.1 Información General del DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Machine failure          10000 non-null  int64  
 9   TWF                      10000 non-null  int64  
 10  HDF                      10000 non-null  int64  
 11  PWF                      10000 non-null  int64  
 12  OSF                      10000 non-null  int64  
 13  RNF                      100

Paso 3: Preprocesamiento de Datos y Escalado

1. Eliminar Columnas Innecesarias: Se eliminan las columnas de identificación (UDI, Product ID) que no tienen poder predictivo.

2. Separar X e y: El dataset se divide en características predictoras (X) y la variable objetivo (y), que es 'Machine failure'.

3. Codificación de Variable Categórica: Se aplica One-Hot Encoding a la columna 'Type' (L, M, H) para convertirla en columnas binarias (ej: Type_L, Type_M), ya que los modelos matemáticos solo trabajan con números.

4. Identificar Numéricas: Se identifican las columnas con valores continuos (temperatura, velocidad, torque, etc.).

5. Escalado (StandardScaler): Se aplica StandardScaler a las columnas numéricas. Esto es crucial: centra los datos en una media de 0 y desviación estándar de 1, igualando las escalas para que Naive Bayes y KDE funcionen correctamente.

6. División de Datos (70/30): El dataset se divide en un 70% para entrenamiento (X_train, y_train) y un 30% para prueba (X_test, y_test). La opción stratify=y asegura que la proporción de fallos (el desbalance) se mantenga igual en ambos conjuntos.

In [3]:
# Importar librerías necesarias
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Volver a cargar el dataframe original y limpiar nombres
df = pd.read_csv("./ai4i2020.csv") # Asegúrate de que esta ruta sea correcta
def limpiar_nombre_columna(col):
    col = ' '.join(col.split())
    return col.strip()
df.columns = [limpiar_nombre_columna(col) for col in df.columns]
NOMBRE_OBJETIVO = 'Machine failure'

# 3.1 Columnas a eliminar AHORA:
columnas_a_eliminar = [
    'UDI', 
    'Product ID', 
    # ¡NUEVA CORRECCIÓN! Eliminar las causas binarias de fallo
    'TWF', 'HDF', 'PWF', 'OSF', 'RNF'
]
df = df.drop(columns=columnas_a_eliminar)
print(f"Columnas eliminadas: {columnas_a_eliminar}")


# 3.2 Separar X (predictoras) e y (objetivo)
X = df.drop(columns=[NOMBRE_OBJETIVO])
y = df[NOMBRE_OBJETIVO]


# 3.3 Codificación de Variable Categórica ('Type')
# Se mantiene drop_first=True para evitar colinealidad (corregida previamente)
X = pd.get_dummies(X, columns=['Type'], drop_first=True) 
print(f"Columnas finales de X: {X.columns.tolist()}")


# 3.4 Identificar y Escalar columnas numéricas
columnas_numericas = X.select_dtypes(include=np.number).columns.tolist()
scaler = StandardScaler()
X[columnas_numericas] = scaler.fit_transform(X[columnas_numericas])


# 3.5 División en conjuntos de entrenamiento y prueba (70/30)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("\n--- Resumen de la División ---")
print(f"Tamaño de Entrenamiento (X_train): {X_train.shape}")
print(f"Tamaño de Prueba (X_test): {X_test.shape}")

Columnas eliminadas: ['UDI', 'Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']
Columnas finales de X: ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Type_ L   ', 'Type_ M   ']

--- Resumen de la División ---
Tamaño de Entrenamiento (X_train): (7000, 7)
Tamaño de Prueba (X_test): (3000, 7)


Paso 4: Establecer la Línea Base (IA 1 - GaussianNB)

Este paso implementa tu primera IA (IA 1), que es el modelo Gaussian Naive Bayes estándar de Scikit-learn. Este modelo actúa como el punto de referencia (baseline) con el cual compararás tus implementaciones de Naive Bayes con KDE más adelante.

1. Importar Herramientas: Importar las clases necesarias: GaussianNB (el modelo) y roc_auc_score (la métrica requerida en tu metodología).

2. Instanciar y Entrenar: Se inicializa el modelo GaussianNB y se entrena usando el conjunto de entrenamiento (X_train, y_train) que preparamos en el Paso 3.

3. Predicción de Probabilidades: Se utiliza el modelo para predecir las probabilidades de pertenencia a la clase (0 o 1) en el conjunto de prueba (X_test). La métrica AUC ROC requiere probabilidades, no solo la clase final.

4. Evaluación: Se calcula el valor AUC ROC usando las probabilidades predichas y los valores reales (y_test).

In [4]:
# 4.1 Importar el modelo y la métrica de evaluación
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

# 4.2 Instanciar y Entrenar el modelo
gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)

# 4.3 Predecir las probabilidades en el conjunto de prueba
# Usamos predict_proba para obtener la probabilidad de que sea la clase 1 (fallo)
y_pred_proba_gnb = gnb_model.predict_proba(X_test)[:, 1]

# 4.4 Evaluar el modelo con la métrica AUC ROC
auc_roc_gnb = roc_auc_score(y_test, y_pred_proba_gnb)

print("--- Evaluación de la Línea Base (Gaussian Naive Bayes) ---")
print(f"AUC ROC del Baseline (GaussianNB): {auc_roc_gnb:.4f}")

# Guardamos el resultado del baseline para compararlo en pasos futuros
resultados = {'GaussianNB_Baseline': auc_roc_gnb}

--- Evaluación de la Línea Base (Gaussian Naive Bayes) ---
AUC ROC del Baseline (GaussianNB): 0.8626


Paso 5: Implementación del Clasificador Naive Bayes Personalizado

Este código implementa la lógica de Naive Bayes utilizando el logaritmo de las probabilidades para evitar errores de underflow (probabilidades muy pequeñas). Usaremos KernelDensity de Scikit-learn como la herramienta para la estimación.

1. Definir la Clase: Creamos una clase CustomNaiveBayesKDE con un método de inicialización (__init__) para almacenar el método KDE y el bandwidth $h$ que se usarán.
2. Método fit(X, y) (Entrenamiento):
- Calcular Probabilidad a Priori: Calcula $P(y)$, la probabilidad de cada clase (Fallo y No Fallo).
- Entrenar KDE: Para cada característica $x_i$ y para cada clase $y$, ajusta el estimador de densidad (KDE) a los datos de entrenamiento. Estos modelos ajustados se almacenan.
3. Método predict_proba(X) (Predicción):
- Calcular Verosimilitud: Para una nueva muestra de prueba, usa los modelos KDE entrenados para calcular la verosimilitud (probabilidad de las características dadas las clases), $P(x_i \mid y)$.
- Aplicar la Regla de Bayes: Combina la Probabilidad a Priori ($P(y)$) con la Verosimilitud para obtener la probabilidad final de pertenencia a la clase.

In [13]:
# =================================================================
# PASO 5: IMPLEMENTACIÓN DEL CLASIFICADOR NAIVE BAYES PERSONALIZADO
# (INCLUYE CORRECCIÓN DE SUAVIZADO/CLIPPING PARA EVITAR ERRORES NaN EN AUC ROC)
# =================================================================

import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.base import BaseEstimator, ClassifierMixin

# Definimos el valor de suavizado para el clipping
SMOOTHING_CLIP = 1e-10

class CustomNaiveBayesKDE(BaseEstimator, ClassifierMixin):
    
    def __init__(self, kernel='gaussian', bandwidth=1.0):
        # Almacena el tipo de kernel y el bandwidth (h)
        self.kernel = kernel
        self.bandwidth = bandwidth
        
        # Inicializa las variables que se almacenarán durante el entrenamiento
        self.classes_ = None
        self.class_log_prior_ = None
        self.feature_estimators_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_features = X.shape[1]
        
        # Inicializar estructuras para almacenar las probabilidades a priori y los estimadores de densidad
        self.class_log_prior_ = {}
        self.feature_estimators_ = {}
        
        # 1. Calcular Probabilidad a Priori (P(y))
        for c in self.classes_:
            # Obtener índices de las muestras que pertenecen a la clase actual
            X_c = X[y == c]
            
            # Calcular log(P(y))
            self.class_log_prior_[c] = np.log(X_c.shape[0] / X.shape[0])
            
            # 2. Entrenar KDE para cada característica (P(x_i | y))
            self.feature_estimators_[c] = {}
            for i in range(n_features):
                # Extraer la característica i para la clase c
                feature_data = X_c.iloc[:, i].values.reshape(-1, 1)
                
                # Instanciar y entrenar el KDE con el kernel y bandwidth definidos
                kde = KernelDensity(kernel=self.kernel, bandwidth=self.bandwidth)
                kde.fit(feature_data)
                self.feature_estimators_[c][i] = kde

        return self

    def predict_proba(self, X):
        n_samples = X.shape[0]
        # Inicializar matriz de log-probabilidades (una fila por muestra, una columna por clase)
        log_prob_matrix = np.zeros((n_samples, len(self.classes_)))
        
        # Calcular log(P(y | x)) para cada clase
        for idx, c in enumerate(self.classes_):
            # Sumar el log-prior (log(P(y)))
            log_prob_matrix[:, idx] = self.class_log_prior_[c]
            
            # Sumar las log-verosimilitudes (log(P(x_i | y))) para cada característica
            n_features = X.shape[1]
            for i in range(n_features):
                feature_data = X.iloc[:, i].values.reshape(-1, 1)
                
                # Obtener log-verosimilitud del KDE
                log_likelihoods = self.feature_estimators_[c][i].score_samples(feature_data)
                
                # log(P(y | x)) = log(P(y)) + SUM(log(P(x_i | y)))
                log_prob_matrix[:, idx] += log_likelihoods

        # 3. Convertir log-probabilidades a probabilidades normales
        max_log_prob = np.max(log_prob_matrix, axis=1, keepdims=True)
        exp_prob = np.exp(log_prob_matrix - max_log_prob)
        
        # Normalizar para que las probabilidades sumen 1
        prob_matrix = exp_prob / np.sum(exp_prob, axis=1, keepdims=True)
        
        # 🚀 APLICAR CLIPPING: Asegurar que las probabilidades no sean 0 o 1 exacto
        prob_matrix = np.clip(prob_matrix, SMOOTHING_CLIP, 1 - SMOOTHING_CLIP)
        
        return prob_matrix

    def predict(self, X):
        # La predicción es simplemente la clase con la probabilidad más alta
        probabilities = self.predict_proba(X)
        return self.classes_[np.argmax(probabilities, axis=1)]


print("Clase CustomNaiveBayesKDE definida exitosamente con suavizado (clipping).")

Clase CustomNaiveBayesKDE definida exitosamente con suavizado (clipping).


Paso 6: KDE con Regla de Silverman (Método 2.c)

Dado que la librería scipy.stats.gaussian_kde maneja el cálculo de la densidad de manera diferente a sklearn.neighbors.KernelDensity, crearemos una función de entrenamiento separada.

1. Importar Herramienta: Importar gaussian_kde de SciPy, que se basa en la Regla de Silverman.
2. Definir CustomNaiveBayesKDE_Silverman: Creamos una nueva clase que hereda la lógica general del Naive Bayes implementado en el Paso 5, pero que sobrescribe el método de entrenamiento (fit) para usar la función de SciPy.
3. Entrenamiento con Silverman: Dentro del fit, usamos gaussian_kde(feature_data) para entrenar la estimación de densidad para cada característica y clase. Esta función calcula internamente el $h$ óptimo con la Regla de Silverman.
4. Cálculo de Verosimilitud: Para la predicción, usamos el método .logpdf() del objeto de SciPy para obtener el $\log P(x_i \mid y)$.
5. Evaluación: Entrenar y evaluar esta nueva clase para obtener el AUC ROC y compararlo con el Baseline (Paso 4).

In [14]:
# =================================================================
# PASO 6 (CORREGIDO): IMPLEMENTACIÓN Y EVALUACIÓN DE KDE CON REGLA DE SILVERMAN
# Solución al TypeError forzando la conversión a float.
# =================================================================

from scipy.stats import gaussian_kde
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin # Importar para que la herencia funcione
# Asume que CustomNaiveBayesKDE ya está definida del Paso 5

class CustomNaiveBayesKDE_Silverman(CustomNaiveBayesKDE):
    
    # 6.1 Implementación de FIT usando la Regla de Silverman (SciPy)
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_features = X.shape[1]
        self.class_log_prior_ = {}
        self.feature_estimators_ = {}
        
        for c in self.classes_:
            X_c = X[y == c]
            
            # Cálculo del log(P(y)) - Probabilidad a Priori
            self.class_log_prior_[c] = np.log(X_c.shape[0] / X.shape[0])
            self.feature_estimators_[c] = {}
            
            # Entrenamiento de KDE para cada característica usando SciPy
            for i in range(n_features):
                # CORRECCIÓN CLAVE 1: Forzar la conversión a numpy array de flotantes
                feature_data = X_c.iloc[:, i].values.astype(np.float64)
                
                # gaussian_kde calcula el bandwidth h automáticamente con la Regla de Silverman
                # Se requiere que feature_data sea 1D para SciPy
                kde = gaussian_kde(feature_data) 
                self.feature_estimators_[c][i] = kde
        
        return self

    # 6.2 Implementación de PREDICT_PROBA adaptada a SciPy
    def predict_proba(self, X):
        n_samples = X.shape[0]
        log_prob_matrix = np.zeros((n_samples, len(self.classes_)))
        
        for idx, c in enumerate(self.classes_):
            # Sumar el log-prior (log(P(y)))
            log_prob_matrix[:, idx] = self.class_log_prior_[c]
            
            n_features = X.shape[1]
            for i in range(n_features):
                # CORRECCIÓN CLAVE 2: Forzar la conversión a numpy array de flotantes
                feature_data = X.iloc[:, i].values.astype(np.float64)

                # Usar logpdf() de SciPy para obtener la log-verosimilitud
                log_likelihoods = self.feature_estimators_[c][i].logpdf(feature_data)
                
                log_prob_matrix[:, idx] += log_likelihoods

        # Conversión de log-probabilidades a probabilidades (código estándar del Paso 5)
        max_log_prob = np.max(log_prob_matrix, axis=1, keepdims=True)
        exp_prob = np.exp(log_prob_matrix - max_log_prob)
        prob_matrix = exp_prob / np.sum(exp_prob, axis=1, keepdims=True)
        
        return prob_matrix

# 6.3 Evaluación del Modelo con Regla de Silverman
modelo_silverman = CustomNaiveBayesKDE_Silverman()
# Asume que X_train y y_train están listos del Paso 3
modelo_silverman.fit(X_train, y_train) 

# Asume que X_test y y_test están listos del Paso 3
y_pred_proba_silverman = modelo_silverman.predict_proba(X_test)[:, 1]
auc_roc_silverman = roc_auc_score(y_test, y_pred_proba_silverman)

print("--- Evaluación de KDE con Regla de Silverman ---")
print(f"AUC ROC (KDE Silverman): {auc_roc_silverman:.4f}")

# Almacenar el resultado (asume que 'resultados' existe del Paso 4)
# Asegúrate de que la variable resultados existe y tiene el GNB baseline
# resultados['KDE_Silverman'] = auc_roc_silverman
# print("\nResultados actuales para comparación:")
# print(pd.Series(resultados))

--- Evaluación de KDE con Regla de Silverman ---
AUC ROC (KDE Silverman): 0.9134


Paso 7: KDE Tipo Parzen (Ventanas Simples)
Este paso implementará los métodos de KDE Tipo Parzen (ventanas simples: rectangular/tophat y triangular/linear) con un bandwidth $h$ fijo, tal como lo pide la Metodología (Punto 2.b).
Utilizaremos la clase base CustomNaiveBayesKDE que creamos en el Paso 5, ya que esa clase está diseñada para usar sklearn.neighbors.KernelDensity, la cual soporta estos kernels simples y requiere un $h$ fijo.
1. Modelo Parzen (Tophat): Instanciar CustomNaiveBayesKDE con kernel='tophat' y un bandwidth $h$ fijo (ej., h=0.5).
2. Entrenar y Evaluar (Tophat): Entrenar el modelo con $X_{train}$ y calcular su AUC ROC en $X_{test}$.
3. Modelo Parzen (Linear): Instanciar el modelo con kernel='linear' y el mismo bandwidth $h$ fijo (ej., h=0.5).
4. Entrenar y Evaluar (Linear): Entrenar y calcular su AUC ROC

In [15]:
from sklearn.metrics import roc_auc_score
import pandas as pd

# Definimos un bandwidth fijo para el método Parzen (este valor es arbitrario para empezar)
H_PARZEN_FIJO = 0.5 
print(f"Usando bandwidth fijo (h) = {H_PARZEN_FIJO} para métodos Parzen.")

# --- 7.1 Modelo KDE Parzen con Kernel Rectangular (Tophat) ---
print("\n--- 7.1 Evaluación KDE Parzen (Tophat) ---")
modelo_parzen_tophat = CustomNaiveBayesKDE(kernel='tophat', bandwidth=H_PARZEN_FIJO)
modelo_parzen_tophat.fit(X_train, y_train)

y_pred_proba_tophat = modelo_parzen_tophat.predict_proba(X_test)[:, 1]
auc_roc_tophat = roc_auc_score(y_test, y_pred_proba_tophat)

print(f"AUC ROC (KDE Tophat, h={H_PARZEN_FIJO}): {auc_roc_tophat:.4f}")
resultados['KDE_Parzen_Tophat'] = auc_roc_tophat


# --- 7.2 Modelo KDE Parzen con Kernel Triangular (Linear) ---
print("\n--- 7.2 Evaluación KDE Parzen (Linear) ---")
modelo_parzen_linear = CustomNaiveBayesKDE(kernel='linear', bandwidth=H_PARZEN_FIJO)
modelo_parzen_linear.fit(X_train, y_train)

y_pred_proba_linear = modelo_parzen_linear.predict_proba(X_test)[:, 1]
auc_roc_linear = roc_auc_score(y_test, y_pred_proba_linear)

print(f"AUC ROC (KDE Linear, h={H_PARZEN_FIJO}): {auc_roc_linear:.4f}")
resultados['KDE_Parzen_Linear'] = auc_roc_linear


# --- 7.3 Resumen de Resultados ---
print("\n--- Resumen de Resultados Acumulados ---")
print(pd.Series(resultados).sort_values(ascending=False))

Usando bandwidth fijo (h) = 0.5 para métodos Parzen.

--- 7.1 Evaluación KDE Parzen (Tophat) ---
AUC ROC (KDE Tophat, h=0.5): 0.9115

--- 7.2 Evaluación KDE Parzen (Linear) ---
AUC ROC (KDE Linear, h=0.5): 0.9156

--- Resumen de Resultados Acumulados ---
KDE_Parzen_Linear           0.915594
KDE_Gaussiano_Optimizado    0.913236
KDE_Parzen_Tophat           0.911450
GaussianNB_Baseline         0.862633
dtype: float64


8: KDE Gaussiano con Optimización Bayesiana

Este paso implementa el KDE Gaussiano (Método 2.a), pero en lugar de usar un bandwidth ($h$) fijo, utilizaremos una técnica de búsqueda avanzada (GridSearchCV de Scikit-learn) para encontrar el mejor $h$ que maximice el rendimiento (AUC ROC).

1. Definir la Búsqueda: Definir el rango de valores (el grid) para el bandwidth $h$ que queremos probar (ej., $0.05$ a $2.0$).
2. Configurar GridSearchCV: Utilizar GridSearchCV junto con tu clase CustomNaiveBayesKDE para buscar el mejor $h$. GridSearchCV automáticamente entrena y evalúa tu clasificador para cada valor de $h$ en el grid
3. Métrica de Búsqueda: Especificar que la métrica a optimizar debe ser 'roc_auc' (AUC ROC).
4. Entrenar y Encontrar $h$: Entrenar el GridSearchCV usando los datos de entrenamiento. Esto es el proceso de optimización.
5. Evaluar el Mejor Modelo: Tomar el mejor modelo encontrado (el que usa el $h$ óptimo) y evaluarlo en el conjunto de prueba (X_test) final.

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd

print("--- 8.1 Optimizando el Bandwidth (h) para KDE Gaussiano ---")

# 8.2 Definir el rango de valores de 'h' (bandwidth) a probar
# Es crucial probar un rango apropiado. Usamos una distribución logarítmica.
# Esto toma tiempo, por eso el rango es limitado.
param_grid = {
    'bandwidth': np.logspace(-1, 0.3, 15) # Rango de h desde ~0.1 hasta ~2.0
}

# 8.3 Configurar GridSearchCV para buscar el mejor 'h'
# Instanciamos tu clasificador personalizado con el kernel Gaussiano
nb_kde_gauss = CustomNaiveBayesKDE(kernel='gaussian')

# Instanciar el GridSearchCV
# scoring='roc_auc' le dice que optimice usando el AUC ROC
# cv=5 usa 5-fold Cross-Validation en el conjunto de entrenamiento
grid_search = GridSearchCV(
    nb_kde_gauss,
    param_grid,
    scoring='roc_auc',
    cv=5, 
    verbose=1,
    n_jobs=-1 # Usar todos los núcleos del procesador
)

# 8.4 Entrenar y encontrar el h óptimo
# Esto toma tiempo ya que entrena 15 valores * 5 folds = 75 modelos.
grid_search.fit(X_train, y_train)

# 8.5 Obtener el mejor resultado y evaluarlo en X_test
best_h = grid_search.best_params_['bandwidth']
best_model_opt = grid_search.best_estimator_

# Predecir las probabilidades con el modelo óptimo
y_pred_proba_opt = best_model_opt.predict_proba(X_test)[:, 1]
auc_roc_opt = roc_auc_score(y_test, y_pred_proba_opt)

print(f"\n--- 8.6 Evaluación Final ---")
print(f"El mejor bandwidth (h) encontrado es: {best_h:.4f}")
print(f"AUC ROC (KDE Gaussiano Optimizado): {auc_roc_opt:.4f}")

# Almacenar el resultado final
resultados['KDE_Gaussiano_Optimizado'] = auc_roc_opt




--- 8.1 Optimizando el Bandwidth (h) para KDE Gaussiano ---
Fitting 5 folds for each of 15 candidates, totalling 75 fits


c:\Users\kenet\Documents\keneth_bracamonte_clase_IA\.venv\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(



--- 8.6 Evaluación Final ---
El mejor bandwidth (h) encontrado es: 0.1000
AUC ROC (KDE Gaussiano Optimizado): 0.9132


Paso 9: Imprimir resultado

En este paso se imprimen los resultado de los modelos

In [ ]:
print("\n--- PASO 9: RESUMEN FINAL DE LA METODOLOGÍA ---")
resultados_df = pd.Series(resultados).sort_values(ascending=False).to_frame(name='AUC_ROC')

print("\nResultados de los 4 Modelos de Naive Bayes:")
print(resultados_df)


--- PASO 9: RESUMEN FINAL DE LA METODOLOGÍA ---

Resultados de los 4 Modelos de Naive Bayes:
                           AUC_ROC
KDE_Parzen_Linear         0.915594
KDE_Gaussiano_Optimizado  0.913236
KDE_Parzen_Tophat         0.911450
GaussianNB_Baseline       0.862633
